<a href="https://colab.research.google.com/github/Mel-iza/Weni_XP_IA/blob/main/notebook/weni_xp_hands_on_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Exploração do problema

<img src= "https://user-images.githubusercontent.com/72058182/226016150-eaa9fde8-3ecf-47e8-aa7a-c6d60ea9c632.png">

<h3><b>Problema de negócio</b></h3> 
Como identificar se uma mensagem é de um bot ou humano? <br><br>
Dificuldade em identificar esse caso se tratando de uma situação, tanto com contexto como sem, pode ser uma tarefa complicada. <br>
Em alguns casos, a consequência disso pode ser: <br>

* Desinformação e propagação de notícias falsas;
* Perdas financeiras ou de recursos pessoais/humanos;
 


### 1. Instalação das bibliotecas e importação dos dados 


Instalação das bibliotecas que vamos utilizar:

In [ ]:
# Instalação das bibliotecas necessárias

!pip install pandas==1.5.3
!pip install -U scikit-learn
!pip install numpy==1.24.2
!pip install scipy==1.2.1 
!pip install Unidecode==1.3.6
!pip install nltk==3.8.1
!pip install spacy==3.5.0
!python -m spacy download pt_core_news_sm

Caso você tenha algum erro com a instalação das bibliotecas, pode desinstalar executando esse código em uma célula. <br>

```
!pip uninstall scikit-learn
!pip uninstall numpy
!pip uninstall pandas
!pip uninstall scipy
```
Em seguida, reinicie o ambiente de execução. Clique na barra de tarefas na opção `ambiente de execução` ↪ `reiniciar ambiente de execução`.

Se der tudo certinho, pode ignorar esse aviso e seguir em frente ✌

<b>Importação das bibliotecas e módulos</b>

In [2]:
# Manipulação de dados
import pandas as pd
import numpy as np
import unicodedata
import unidecode
import re


# Machine Learning, vetorização e métricas
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV


# Tratamento e manipulação de texto
import nltk
import spacy
import pt_core_news_sm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

import warnings
warnings.simplefilter("ignore")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


<b>Importando os dados</b>

1) Importando o conjunto de dados com arquivo direto na sessão do notebook.<br>

<i>Clique no ícone de pasta na barra lateral esquerda do Google Colab</i>

<img src= "https://user-images.githubusercontent.com/72058182/226423528-8da8377f-5643-4e4b-8dc9-860eebe32352.png" height="250px">


<i>Em seguida clique no ícone para fazer upload de arquivo e escolha o arquivo `train_dataset.csv` que você já fez download previamente pelo repositório ou drive.</i>

<img src= "https://user-images.githubusercontent.com/72058182/226424068-843ca16e-4248-45b8-b72e-dfe5dd15e86b.png" height="220px">

<i>Em seguida, assim que o arquivo estiver carregado na sessão é só executar a célula abaixo</i>

Também é possível abrir o dataset por meio do Google Colab ou o próprio Github. Fique à vontade para utilizar o meio que mais se sinta confortável.


In [5]:
# Importando o conjunto de dados com arquivo direto na sessão do notebook

data =  pd.read_csv('/content/train_dataset.csv')
data

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot
...,...,...
1537,42,humano
1538,start,humano
1539,Quais são os riscos? 😷,humano
1540,Nao,humano


----------------

### 2. Conhecer e pré-processar os dados


Conferindo algumas informações básicas sobre o conjunto de dados.<br>

<i>Com o método `info()` é possível saber várias informações sobre o nosso conjunto de dados: quantidade de colunas, linhas, tipos de cada um, quantidade de dados nulos, uso de memória entre outros. E com o `shape` conseguimos ver a quantidade de linhas e colunas </i>

In [6]:
# Mostrando algumas informações básicas do conjunto de dados

print('linhas e colunas: ', data.shape)
print('\n')
data.info()

linhas e colunas:  (1542, 2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1542 non-null   object
 1   label   1542 non-null   object
dtypes: object(2)
memory usage: 24.2+ KB


Conferindo algumas linhas do conjunto de dados:
- primeiras linhas
- linhas escolhidas aleatoriamente

In [7]:
# Mostrando as primeiras linhas do conjunto de dados

data.head()

,text,label
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot
4,Pera só um instante que vou checar aqui se enc...,chatbot


In [8]:
# Mostrando algumas linhas do conjunto de dados de forma aleatoria

data.sample(10)

,text,label
1479,Isso,humano
1427,start,humano
775,Certo,humano
1116,Ou,humano
1014,Sim aparece,humano
728,tenho chance de ter pego?,humano
913,1,humano
1495,Sim,humano
717,Acho que esse,humano
64,"Victor, deseja realizar a autoavaliação do nov...",chatbot


Vamos verificar a quantidade de dados nulos presente no conjunto de dados?


In [10]:
# Quantidade de dados nulos

data.isnull().sum()

text     0
label    0
dtype: int64

---------------

E agora, como fazer com que a máquina consiga compreender esses textos? Vamos começar pelas técnicas de <b>pré-processamento</b>. <br>

* `padronização do texto`
* `removeção stop-words (a, e, de, o, etc)`
* `tokenização`

<i>Padronização, remoção de stop-words e vetorização</i>

In [11]:
# Função para remover acentos e números

def remover_acentos_e_numeros(txt):
    nfkd = unicodedata.normalize('NFKD', txt)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub(r"(@[A-Za]+)|([^A-Za-z \t])|(\w+:\/\/\S+)| ^rt|http.+?","", palavraSemAcento)
    palavraSemAcento = str(palavraSemAcento).lower()
    palavraSemAcento = re.sub(r"\d+", "", palavraSemAcento)
    palavraSemAcento = re.sub(r'  ', ' ', palavraSemAcento)
    palavraSemAcento = re.sub(r'compartilheversao para impressao comentarios',' ',
                                palavraSemAcento)
    return palavraSemAcento

In [12]:
# Aplicando a função para padronização em uma nova coluna. Observe a diferença das colunas 'text' e 'texto_padronizado'.

data['texto_padronizado'] = [remover_acentos_e_numeros(i) for i in data['text']]
data.head()

,text,label,texto_padronizado
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...
5,Você tem algum desses problemas de saúde?\n\n1...,chatbot,voce tem algum desses problemas de saude diabe...
6,Infelizmente eu não consegui te entender. Você...,chatbot,infelizmente eu nao consegui te entender voce ...
7,Qual o seu primeiro nome?,chatbot,qual o seu primeiro nome
8,Você não possui nenhum pedido em aberto!,chatbot,voce nao possui nenhum pedido em aberto
9,Você está com algum dos sintomas abaixo?\n\n1....,chatbot,voce esta com algum dos sintomas abaixo febre ...


In [14]:
# Remover as stop words com NLTK

from nltk import download
download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])


from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Por curiosidade, vamos dar uma olhadinha em quais são aquelas palavras que podemos tirar do texto sem muito medo porque elas não acrescentam significado?

In [17]:
# Conhecendo as stop words

print(stop_words)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [19]:
# Aplicando a remoção de stop-words na coluna padronizada 

data['texto_clean'] = data['texto_padronizado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data.head()

,text,label,texto_padronizado,texto_clean
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...,ola prazer assistente virtual papel ajudar ven...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...,fechado vamos fazer assim voce passa dados ja ...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...,pra adiantar vou precisar voce fale email
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...,humm nao entendi desculpadigite falar consulto...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...,pera so instante vou checar aqui encontro algu...


**Outra forma de remover as stop words - usando txt**. <br>
Essa parte é mais uma curiosidade, é uma possibilidade legal para sabermos que existe. <br>
Como já removemos as stop words na célula de cima, não precisamos fazer de novo. É apenas um método diferente para alcançar o mesmo resultado.

Caso possua curiosidade de testar depois, é só importar o arquivo `stopwords-pt.txt` na sessão do Colab (como fizemos com o dataset) e executar a célula abaixo.

In [20]:
#@title # Outra forma de remover as stop words - usando txt

stop_portugues = list(open(r'stopwords-pt.txt', encoding="UTF-8"))
stop_portugues= [i.replace('\n','') for i in stop_portugues] 
stop_portugues = [remover_acentos_e_numeros(i) for i in stop_portugues]

def remover_stop_words(x):
    return [w for w in x if w not in stop_portugues]

data['texto_clean_1'] = data['texto_padronizado'].str.split().apply(remover_stop_words).str.join(' ')
data.head()    

,text,label,texto_padronizado,texto_clean,texto_clean_1
0,"Olá, muito prazer! Sou o seu assistente virtua...",chatbot,ola muito prazer sou o seu assistente virtual ...,ola prazer assistente virtual papel ajudar ven...,ola prazer assistente virtual papel ajudar ven...
1,"Fechado, vamos fazer assim. Você passa os dado...",chatbot,fechado vamos fazer assim voce passa os dados ...,fechado vamos fazer assim voce passa dados ja ...,fechado vamos passa dados deixo dei cadastro p...
2,"Pra adiantar, vou precisar que você me fale qu...",chatbot,pra adiantar vou precisar que voce me fale qua...,pra adiantar vou precisar voce fale email,adiantar vou precisar fale email
3,"Humm eu não entendi, desculpa.\n\nDigite 1 par...",chatbot,humm eu nao entendi desculpadigite para falar ...,humm nao entendi desculpadigite falar consulto...,humm entendi desculpadigite falar consultordig...
4,Pera só um instante que vou checar aqui se enc...,chatbot,pera so um instante que vou checar aqui se enc...,pera so instante vou checar aqui encontro algu...,pera instante vou checar encontro algum pedido...


In [21]:
# Vetorizando o texto e aplicando TF-idf

tfidf = TfidfVectorizer(stop_words = stop_words)

X = tfidf.fit_transform(data.texto_clean)
y = (data.label)

### 3. Vamos modelar? Criação de um modelo de classificação

In [22]:
# Definindo modelos que serão testados - Regressão Logística

modelos = {"Regressão Logística": LogisticRegressionCV()}

Para fazer o treinamento do nosso modelo precisamos separar o texto que ele vai ter que treinar e as labels/targets. Fazendo uma analogia, o texto seria como todas as referências que utilizamos para estudar para uma prova por exemplo e as labels seriam o gabarito. <br>

Não faz sentido a gente ir estudar para uma prova com o gabarito em mãos, certo? É exatamente esse mesmo raciocínio que estamos aplicando aqui. Para o modelo não 'colar' estamos tirando o gabarito dele, assim ele vai aprender apenas os padrões das mensagens. 

In [23]:
# Separando o texto em features e targets

X = data.texto_clean
Y = data.label

In [24]:
# Instanciando o CountVectorizer 

tokenizer = CountVectorizer(ngram_range=(1,1))

# Tokenizando os dados 
tokenizer.fit(X)
X_mat = tokenizer.transform(X)
X_df = pd.DataFrame(X_mat.todense())

In [25]:
from sklearn.model_selection import cross_val_score

# Transformando o y em número para que o modelo consiga avaliar a classe. Escolhi o valor 0 para chatbot e 1 para humano
y = y.replace(['chatbot', 'humano'], [0, 1]).astype('category')


# Cross validation para avaliar a acurácia dos modelos.
for nome, modelo in modelos.items():
    accuracy = cross_val_score(modelo, X_df, y, cv = 3, scoring='accuracy').mean() # Definindo 3 folds e fazendo a média
    print(nome, accuracy)


Regressão Logística 0.9468223086900128


### 4. Como saber se nosso modelo foi bem? Interpretação de métricas.

A acurácia nos retorna os elementos classificados corretamente pelo modelo.
Ela é uma métrica interessante, pois nos traz o quanto o modelo consegue classificar corretamente.

In [26]:
# Criando dicionário e Avaliando o modelo
dict_metrica = {}
dict_metrica["Tecnica Utilizada"] = ["Vetorização + Remoção de stopwords"]

# Transformando o y em número para que o modelo consiga avaliar a classe. Escolhi o valor 0 para chatbot e 1 para humano
y = y.replace(['chatbot', 'humano'], [0, 1]).astype('category')

# Cross validation para avaliar a acurácia dos modelos.
for nome, modelo in modelos.items():
    accuracy = cross_val_score(modelo, X_df, y, cv = 3, scoring='accuracy').mean() # Definindo 3 folds e fazendo a média
    dict_metrica[nome] = [round(accuracy,4)]
    
# Mostrando métricas
df_metricas = pd.DataFrame(dict_metrica)
df_metricas

,Tecnica Utilizada,Regressão Logística
0,Vetorização + Remoção de stopwords,0.9468


O quão bem o nosso modelo consegue classificar corretamente as classes? Ele consegue classificar corretamente 94% das amostras! <br>



*Alguns pontos importantes*
- A validação cruzada (cross validation - uma técnica para auxiliar a capacidade preditiva do modelo dado uma variabilidade maior) foi aplicada em todos os resultados. Ou seja, eles trazem uma média de 3 folds - cada fold divide aleatoriamente a base de dados.<br>
- Os n-grams utilizados foram de 1 token para todos os modelos <br>
- todos os treinos foram realizados com a coluna textual padronizada,  e sem as stopwords.<br>
- Para outro momento valeria muito utilizar outras métricas como Recall, F1-Score e matriz de confusão para uma avaliação mais completa.

### 5. Se o modelo se saiu bem até agora significa que ele se sairá bem com novos dados?

#### Testando em novos dados


<br> Apesar do viés de seleção, por eu ter acabado de ler alguns tipos de mensagens no dataset, em um arquivo eu escrevi 10 mensagens como se eu fosse atendente e quero ver como o modelo poderia classificar ele.

In [ ]:
df_pred = pd.read_excel("dados_teste.xlsx")
df_pred.head()

In [ ]:
# Separando o texto em features e targets

X = df_pred.text
Y = df.label

In [ ]:
# Instanciando o CountVectorizer 

tokenizer = CountVectorizer(ngram_range=(1,1))

# instanciando e treinando o modelo

tokenizer.fit(X)
df_treino = tokenizer.transform(X)
df_teste = tokenizer.transform(df_pred.text)

In [ ]:
# modelando os dados e treinando

regressao = LogisticRegressionCV(random_state=123)

modelo_novo = regressao.fit(df_treino, df.label.sample(10))

y_pred = modelo_novo.predict(df_teste)
df_pred['y_pred'] = y_pred
df_pred

In [ ]:
# Vamos verificar a probabilidade da classificação pra ver com quanta confiança ele classificou as mensagens?

y_pred_prob = modelo_novo.predict_proba(df_teste)

y_pred_hum = []
y_pred_chat = []
for i in y_pred_prob:
    y_pred_hum.append(i[1])
    y_pred_chat.append(i[0])
    
df_pred['y_pred_hum'] = y_pred_hum
df_pred['y_pred_chat'] = y_pred_chat

df_pred